# Portfolio Analyzer

The first segment of this application will allow you to enter the stocks or bonds within your portfolio and their quantities, and returns the portfolio's value.  A simulation is then run on the portfolio to determine the most likely possible outcomes.  You can then compare your portfolio to the wider market.

In [88]:
# Imports required libraries
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import hvplot.pandas
from pathlib import Path
import questionary

%matplotlib inline

In [2]:
load_dotenv()

True

In [3]:
# Locates and stores keys
alpaca_api = os.getenv("ALPACA_API_KEY")
alpaca_secret = os.getenv("ALPACA_SECRET_KEY")
type(alpaca_api)
type(alpaca_secret)

# Creates the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api,
    alpaca_secret,
    api_version="v2"
)

In [4]:
# Sets timeframe to 1Day
timeframe = "1Day"

# Formats current date as ISO format
# You can choose to set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you could select any arbitrary start/end dates
start_date = pd.Timestamp("2022-05-12", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-05-12", tz="America/New_York").isoformat()

# Create your portfolio:

In [5]:
# Creates the list of tickers that the user will input, and allows for their input

tickers = []

tickers.append(input("Enter your first stock or bond ticker (as it appears on the exchange (example: BBBY)) "))
tickers.append(input("Enter your second ticker."))
tickers.append(input("Enter your third ticker."))
tickers.append(input("Enter your fourth ticker."))
tickers.append(input("Enter your fifth ticker."))
print(f"The tickers you have selected are: {tickers}")


Enter your first stock or bond ticker (as it appears on the exchange (example: BBBY))  GME
Enter your second ticker. AMC
Enter your third ticker. MSFT
Enter your fourth ticker. AMZN
Enter your fifth ticker. GOOG


The tickers you have selected are: ['GME', 'AMC', 'MSFT', 'AMZN', 'GOOG']


In [6]:
# Uses the Alpaca get_bars function to get current closing prices each ticker in
# the portfolio and creates a dataframe
portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start_date,
    end_date
).df
    
# Reorganizes the DataFrame
# Separates ticker data
ticker_1 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)
ticker_2 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)
ticker_3 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)
ticker_4 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)
ticker_5 = portfolio[portfolio['symbol']==portfolio.iloc[:,7]].drop('symbol', axis=1)

# Concatenates the ticker DataFrames
portfolio_df = pd.concat(
    [ticker_1, ticker_2, ticker_3, ticker_4, ticker_5],
    axis=1,
    keys=[tickers[0], tickers[1], tickers[2], tickers[3], tickers[4]]
)

# Reviews the Alpaca DataFrame
portfolio_df.head()

GME                                         \
                              open      high      low    close     volume   
timestamp                                                                   
2022-05-12 04:00:00+00:00    10.06    13.710     9.70    11.20  104735519   
2022-05-12 04:00:00+00:00  2055.00  2215.615  2048.11  2138.61    6574160   
2022-05-12 04:00:00+00:00    83.04   108.060    77.77    89.57   10023560   
2022-05-12 04:00:00+00:00  2240.00  2297.130  2202.27  2263.22    2055927   
2022-05-12 04:00:00+00:00   257.32   259.880   250.02   255.35   50675646   

                                                        AMC            \
                          trade_count         vwap     open      high   
timestamp                                                               
2022-05-12 04:00:00+00:00      508133    11.457367    10.06    13.710   
2022-05-12 04:00:00+00:00      358551  2129.391018  2055.00  2215.615   
2022-05-12 04:00:00+00:00      186854    91.085635    83.04   108.060   
2022-05-12 04:00:00+00:00      120731  2250.036001  2240.00  2297.130   
2022-05-12 04:00:00+00:00      709263   254.769493   257.32   259.880   

                                    ...       AMZN                           \
                               low  ...     volume trade_count         vwap   
timestamp                           ...                                       
2022-05-12 04:00:00+00:00     9.70  ...  104735519      508133    11.457367   
2022-05-12 04:00:00+00:00  2048.11  ...    6574160      358551  2129.391018   
2022-05-12 04:00:00+00:00    77.77  ...   10023560      186854    91.085635   
2022-05-12 04:00:00+00:00  2202.27  ...    2055927      120731  2250.036001   
2022-05-12 04:00:00+00:00   250.02  ...   50675646      709263   254.769493   

                              GOOG                                         \
                              open      high      low    close     volume   
timestamp                                                                   
2022-05-12 04:00:00+00:00    10.06    13.710     9.70    11.20  104735519   
2022-05-12 04:00:00+00:00  2055.00  2215.615  2048.11  2138.61    6574160   
2022-05-12 04:00:00+00:00    83.04   108.060    77.77    89.57   10023560   
2022-05-12 04:00:00+00:00  2240.00  2297.130  2202.27  2263.22    2055927   
2022-05-12 04:00:00+00:00   257.32   259.880   250.02   255.35   50675646   

                                                    
                          trade_count         vwap  
timestamp                                           
2022-05-12 04:00:00+00:00      508133    11.457367  
2022-05-12 04:00:00+00:00      358551  2129.391018  
2022-05-12 04:00:00+00:00      186854    91.085635  
2022-05-12 04:00:00+00:00      120731  2250.036001  
2022-05-12 04:00:00+00:00      709263   254.769493  

[5 rows x 35 columns]

In [102]:
# Allows the user to choose whether or not they wish to save their portfolio

file_path = Path("./Resources/portfolio.csv")
answer = input("Would you like to save your portfolio to a CSV file?  Yes or No ")
if answer == ("Yes"):
    portfolio_df.to_csv(file_path)
else:
    print("Continue to the next section.")

Would you like to save your portfolio to a CSV file?  Yes or No  No


Continue to the next section.


In [7]:
# Allows the user to input the quantity of their stocks or bonds
print("What is the quantity of each ticker that you own?")
ticker_1_qty = int(input(f"{tickers[0]}: "))
ticker_2_qty = int(input(f"{tickers[1]}: "))
ticker_3_qty = int(input(f"{tickers[2]}: "))
ticker_4_qty = int(input(f"{tickers[3]}: "))
ticker_5_qty = int(input(f"{tickers[4]}: "))


What is the quantity of each ticker that you own?


GME:  2
AMC:  2
MSFT:  2
AMZN:  2
GOOG:  2


In [8]:
# Fetches the closing prices of each ticker symbol in the portfolio
ticker_1_close = float(portfolio_df.iat[0,3])
ticker_2_close = float(portfolio_df.iat[1,3])
ticker_3_close = float(portfolio_df.iat[2,3])
ticker_4_close = float(portfolio_df.iat[3,3])
ticker_5_close = float(portfolio_df.iat[4,3])

                                    
print(f"Today's closing prices of the tickers in your portfolio are as follows:")
print(f"{tickers[0]} ----- {ticker_1_close}")
print(f"{tickers[1]} ----- {ticker_2_close}")
print(f"{tickers[2]} ----- {ticker_3_close}")
print(f"{tickers[3]} ----- {ticker_4_close}")
print(f"{tickers[4]} ----- {ticker_5_close}")

Today's closing prices of the tickers in your portfolio are as follows:
GME ----- 11.2
AMC ----- 2138.61
MSFT ----- 89.57
AMZN ----- 2263.22
GOOG ----- 255.35


In [107]:
# Calculates the value of each ticker based on quantity owned
ticker_1_value = ticker_1_qty * ticker_1_close
ticker_2_value = ticker_2_qty * ticker_2_close
ticker_3_value = ticker_3_qty * ticker_3_close
ticker_4_value = ticker_4_qty * ticker_3_close
ticker_5_value = ticker_5_qty * ticker_3_close


# Adds up value of entire portfolio
portfolio_value = (ticker_1_value + ticker_2_value + ticker_3_value + ticker_4_value + ticker_5_value)
print(f"Your total portfolio value is ${portfolio_value}")

Your total portfolio value is $4837.040000000001


# Evaluating the S&P 500 index

In [17]:
# Reads the S&P 500 data into a dataframe
sp500 = pd.read_csv(
    Path("Resources/S&P 500 Historical Data (1).csv"),
    index_col="Date",
    parse_dates=True,
    infer_datetime_format=True
)

In [18]:
# Displays the data
display(sp500.head())
display(sp500.tail())

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2022-05-13,"4,024.70","3,975.30","4,039.40","3,963.20",-,2.41%
2022-05-12,"3,930.08","3,903.95","3,964.80","3,858.87",-,-0.13%
2022-05-11,"3,935.18","3,990.08","4,049.09","3,928.82",-,-1.65%
2022-05-10,"4,001.05","4,035.18","4,068.82","3,958.17",-,0.25%
2022-05-09,"3,991.24","4,081.27","4,081.27","3,975.48",-,-3.20%


,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2015-01-08,"2,062.14","2,030.61","2,064.08","2,030.61",-,1.79%
2015-01-07,"2,025.90","2,005.55","2,029.61","2,005.55",-,1.16%
2015-01-06,"2,002.61","2,022.15","2,030.25","1,992.44",-,-0.89%
2015-01-05,"2,020.58","2,054.44","2,054.44","2,017.34",-,-1.83%
2015-01-02,"2,058.20","2,058.90","2,072.36","2,046.04",-,-0.03%


In [19]:
# Checks for null values in the data
sp500.isnull().sum()

Price       0
Open        0
High        0
Low         0
Vol.        0
Change %    0
dtype: int64

In [24]:
# Cleans and prepares the data for the dataframe
sp500.loc[:, 'Open'] = sp500.loc[:, 'Open'].str.replace(",", "")
sp500.loc[:, 'High'] = sp500.loc[:, 'High'].str.replace(",", "")
sp500.loc[:, 'Low'] = sp500.loc[:, 'Low'].str.replace(",", "")
sp500.loc[:, 'Price'] = sp500.loc[:, 'Price'].str.replace(",", "")

# Confirms cleaning of data
sp500.head()

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2022-05-13,4024.70,3975.30,4039.40,3963.20,-,2.41%
2022-05-12,3930.08,3903.95,3964.80,3858.87,-,-0.13%
2022-05-11,3935.18,3990.08,4049.09,3928.82,-,-1.65%
2022-05-10,4001.05,4035.18,4068.82,3958.17,-,0.25%
2022-05-09,3991.24,4081.27,4081.27,3975.48,-,-3.20%


In [49]:
# Changes all data types to float
sp500.loc[:, 'Open'] = sp500.loc[:, 'Open'].astype('float')
sp500.loc[:, 'High'] = sp500.loc[:, 'High'].astype('float')
sp500.loc[:, 'Low'] = sp500.loc[:, 'Low'].astype('float')
sp500.loc[:, 'Price'] = sp500.loc[:, 'Price'].astype('int')

# Confirms data type conversion
sp500.dtypes

Price         int64
Open        float64
High        float64
Low         float64
Vol.         object
Change %     object
dtype: object

In [50]:
# Pulls the closing prices for the index and plots them
sp500_close = sp500['Price'].astype('float')
sp500_close.hvplot(
    ylabel="Price (USD)",
    title="S&P 500 Index: January 2015 - May 2022"
)

:Curve   [Date]   (Price)

In [53]:
# Calculates daily returns for the index and plots them
sp500_daily_returns = sp500_close.pct_change().dropna()
sp500_daily_returns.hvplot(
    title="S&P 500 Daily Returns"
)

:Curve   [Date]   (Price)

# Calculate the daily returns for your portfolio

KeyError: 'open'